In [1]:
#################################################
#Regenerate: Working with MCMC in Gen: Regenerate
#----------------------------------------------
#https://www.gen.dev/Reasoning+About+Regenerate
#################################################

using Gen: bernoulli, @gen, @trace

┌ Info: Precompiling Gen [ea4f424c-a589-11e8-07c0-fd5c91b9da4a]
└ @ Base loading.jl:1260


In [2]:
#The regenerate function is the basis for a MH algorithm in Gen's inference library
@gen function foo(prob_a)
    val = true
    if @trace(bernoulli(prob_a), :a) #Sample a random boolean value
        val = @trace(bernoulli(0.6), :b) && val
    end
    prob_c = val ? 0.9 : 0.2
    val = @trace(bernoulli(prob_c), :c) && val
    return val
end;

In [21]:
using Gen: generate, choicemap, get_choices

#Generate a trace conditioned on the provided choicemap. Weight is the log-probability
trace, weight = generate(foo, (0.3,), choicemap((:a, true), (:b, false), (:c, true)));
choice = get_choices(trace)
display(choice)

│
├── :a : true
│
├── :b : false
│
└── :c : true


In [33]:
using Gen: regenerate, select, NoChange

#Regenerate is like a proposal for a new trace, where select corresponds to the parameters to be updated
(trace, weight, retdiff) = regenerate(trace, (0.3,), (NoChange(),), select(:a));
display(get_choices(trace))

│
├── :a : true
│
├── :b : false
│
└── :c : true


Whereas generate was given the expicit choice map of constraints (u) as an argument, regenerate constructs u by starting with the previous trace t and then removing any selected addresses. In other words, regenerate is like generate, but where the constraints are the choices made in the previous trace less the selected choices.

In [53]:
#One can imagine how this can be used to easily construct a Gibbs sampling algorithm
trace, weight = generate(foo, (0.3,), choicemap((:a, true), (:b, false), (:c, true)));
display(get_choices(trace))
println("weight: $weight");
(trace, weight, retdiff) = regenerate(trace, (0.3,), (NoChange(),), select(:a));
display(get_choices(trace))
println("weight: $weight");

│
├── :a : true
│
├── :b : false
│
└── :c : true


│
├── :a : false
│
└── :c : true


weight: -3.7297014486341915
weight: 1.504077396776274


Exercise: Draw a graph in which each random choice map t where p(t;x) > 0 is a node, and where there are directed edges from t to t' if applying regenerate to t with selection a can produce trace t'. Do the same for selections b and c. What about selection a, b, c?